# Minimum k-cut Algorithm

### High-level Algorithm Specification:

1. Create vertex list and an edges list, e.g.:

    ```javascript
    vertices = {1: [2,4,5], 2: [3,4,5], 3: [2,4], 4: [1,2,3], 5: [1,2]}
    edges = [[1,2], [1,4], [1,5], [2,3], [2,4], [2,5], [3,4]]
    ```

2. Keep track of the minimum cut so far:

    ```javascript
    // really this could be the max degree of all vertices, I believe
    min_edges_so_far = len(edges)
    min_vertex_sets = {1:[], 2:[], 3:[], 4:[], 5:[]}
    ```

3. *Iterate at least `n^2 log n` times (where n is the original number of vertices)*
    
    **Intiate:**
    
    ```javascript
    temp_vertex_sets = copy(min_vertex_sets)
    temp_vertices = copy(vertices)
    temp_edges = copy(edges)
    ```

    **While num_vertices > k:**

    1. Pick an edge at random: the first vertex (`v1`) will absorb the second (`v2`). Add `v2` and `temp_vertex_sets[v2]` to `temp_vertex_sets[v1]` and delete `temp_vertex_sets[v2]`.
    2. All vertices adjacent to `v2` are added to `temp_vertices[v1]` unless already present. Remove `v2` from `temp_vertices[v1]`.
    3. Replace all instances of `v2` in `temp_edges` with `v1`, unless the other vertex of the edge is itself `v1`. In the latter case, delete the edge (e.g. remove self-loops). **Note:** Parallel edges are allowed; there may be multiple instances of an edge comprised the same vertex pair.

    **Finally:** The number of final edges is the number of edges across the final cut in this iteration. If it is less than min_edges_so_far, update `min_edges_so_far = len(temp_edges)` and `min_vertex_sets = temp_vertex_sets`.
        

## Setup: Select all measurements and document ids from the database

In [2]:
import psycopg2
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np
import json

database = 'fomc'
conn = psycopg2.connect("dbname=" + database + " user=abarciauskas")
cur = conn.cursor()

year = 2006
cosine_thresh = 0.2
cur.execute("SELECT Doc1Id,Doc2Id,CosineSimilarity FROM alignments WHERE Year = '" + str(year) + "'"
           " AND CosineSimilarity >= " + str(cosine_thresh))
cosine_sims = cur.fetchall()
len(cosine_sims)

5070

## Step 1: Create the graph

The graph is comprised a list of edges (a vertex tuple) and a dictionary of vertices.

In [223]:
def create_graph(cosine_sims, threshold):
    similar_documents = filter(lambda x: x[2] > threshold, cosine_sims)
    edges = [tuple([x[0],x[1]]) for x in similar_documents]
    vertices = {}
    for edge in edges:
        v1 = edge[0]
        v2 = edge[1]
        if v1 in vertices.keys():
            vertices[v1].add(v2)
        else:
            vertices[v1] = {v2}
        if v2 in vertices.keys():
            vertices[v2].add(v1)
        else:
            vertices[v2] = {v1}
    return [edges, vertices]

edges, vertices = create_graph(cosine_sims, 0.18)

In [224]:
# Testing
# vertices = {1: {2,4,5}, 2: {3,4,5}, 3: {2,4}, 4: {1,2,3}, 5: {1,2}}
# edges = [{1,2}, {1,4}, {1,5}, {2,3}, {2,4}, {2,5}, {3,4}]
print len(vertices)
len(edges)

1172


1959

In [225]:
# need to find disconnected graphs
graphs = []
unvisited = set(vertices.keys())
visited = []

# for every vertex, find all of its connected components and recurse on those vertices
current_vertex = unvisited.pop()
visited.append(current_vertex)
stack_to_visit = list(vertices[current_vertex])
while len(stack_to_visit) > 0:
    current_vertex = stack_to_visit.pop()
    current_adj_vtcs = vertices[current_vertex]
    for v in current_adj_vtcs:
        if v not in visited:
            visited.append(v)
            unvisited.remove(v)
            stack_to_visit.insert(0, v)

graphs.append(visited)
print len(unvisited)
for loner in list(unvisited):
    vertices.pop(loner, None)

edges = [i for j, i in enumerate(edges) if list(i)[0] not in list(unvisited) and list(i)[1] not in list(unvisited)]
print len(vertices.keys())
print len(edges)

62
1110
1924


## Step 2: Keep track of minimum so far

In [178]:
min_edges_so_far = len(edges)
min_vertex_sets = {key: set() for key in vertices.keys()}

## Step 3: Random iterations

In [226]:
import random
import numpy as np
from copy import deepcopy

def run(niters):
    min_edges_so_far = len(edges)
    min_vertex_sets = {key: set() for key in vertices.keys()}
    for iteridx in range(niters):
        if iteridx % 5 == 0: print 'Running iter: ' + str(iteridx)
        temp_vertex_sets = {key: set() for key in vertices.keys()}
        temp_vertices = deepcopy(vertices)
        temp_edges = edges[:]
        while len(temp_vertices) > k:
            # pick an edge at random and delete it
            rand_idx = int(random.random()*len(temp_edges))
            random_edge = temp_edges.pop(rand_idx)
            # Add v2 and temp_vertex_sets[v2] to temp_vertex_sets[v1] and delete temp_vertex_sets[v2]
            v1 = list(random_edge)[0]
            v2 = list(random_edge)[1]
            temp_vertex_sets[v1] = temp_vertex_sets[v1].union(temp_vertex_sets[v2])
            temp_vertex_sets[v1].add(v2)
            temp_vertex_sets.pop(v2, None)

            # All vertices adjacent to v2 are added to temp_vertices[v1] unless already present.
            # Remove v2 from temp_vertices[v1].
            adj_v2 = temp_vertices[v2]
            temp_vertices[v1] = temp_vertices[v1].union(adj_v2)
            temp_vertices[v1].remove(v2)
            temp_vertices.pop(v2, None)

            # Replace all instances of v2 in temp_edges with v1, unless the other vertex of the edge is itself v1.
            # In the latter case, delete the edge (e.g. remove self-loops).
            # Note: Parallel edges are allowed; there may be multiple instances of an edge comprised the same vertex pair.
            remove_edges = []
            for i,cur_edge in enumerate(temp_edges):
                if len(cur_edge) > 1:
                    cur_edge_v1 = list(cur_edge)[0]
                    cur_edge_v2 = list(cur_edge)[1]
                    if (cur_edge == random_edge):
                        remove_edges.append(i)
                    elif cur_edge_v1 == v2:
                        temp_edges[i] = {v1, cur_edge_v2}
                        # remove this edge from temp_vertices
                        # it may have already been removed because we keep parallel edges around
                        if v2 in temp_vertices[cur_edge_v2]: temp_vertices[cur_edge_v2].remove(v2)
                    elif cur_edge_v2 == v2:
                        temp_edges[i] = {cur_edge_v1, v1}
                        # it may have already been removed because we keep parallel edges around
                        if v2 in temp_vertices[cur_edge_v1]: temp_vertices[cur_edge_v1].remove(v2)
            # work around for delete
            temp_edges = [set(i) for j, i in enumerate(temp_edges) if j not in remove_edges]
            #Finally: The number of final edges is the number of edges across the final cut in this iteration.
            #If it is less than min_edges_so_far, update min_edges_so_far = len(temp_edges) and min_vertex_sets = temp_vertex_sets.
            if len(temp_edges) < min_edges_so_far:
                min_edges_so_far = len(temp_edges)
                min_vertex_sets = temp_vertex_sets
        return min_edges_so_far, min_vertex_sets

k = 200
n = len(vertices)
niters = 1

import time

t0 = time.time()
min_edges_so_far, min_vertex_sets = run(niters)
t1 = time.time()

total = t1-t0
print 'Total time for ' + str(niters) + ': ' + str(total)

Running iter: 0
Total time for 1: 1.8028280735


In [227]:
niters = n**2#int(np.ceil(n**2*np.log(n)))
total_seconds = niters*total
minutes = total_seconds/60
hours = minutes/60
print hours/24

25.7090795065


In [228]:
print niters
t0 = time.time()
min_edges_so_far, min_vertex_sets = run(niters)
t1 = time.time()
print 'Total time for ' + str(niters) + ': ' + str(total/60/60) + ' hours'

1232100
Running iter: 0
Total time for 1232100: 0.000500785575973 hours
